In [ ]:
# hide
# skip
!git clone https://github.com/benihime91/gale # install gale on colab
!pip install -e "gale[dev]"

In [ ]:
# default_exp classification.model.heads

In [ ]:
# hide
%load_ext nb_black
%load_ext autoreload
%autoreload 2
%matplotlib inline

<IPython.core.display.Javascript object>

In [ ]:
# hide
import warnings

from nbdev.export import *
from nbdev.showdoc import *

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

# Heads
> A head is a regular `torch.nn.Module` that can be attached to a backbone.


For Image Classification a, `head` typically contains a pooling layer along with the classifier

In [ ]:
# export
import logging
from collections import namedtuple
from dataclasses import dataclass
from typing import *

import torch
import torch.nn.functional as F
from fastcore.all import L, ifnone, store_attr
from omegaconf import MISSING
from timm.models.layers.classifier import _create_fc, _create_pool
from torch import nn

from gale.classification.model.backbones import filter_weight_decay
from gale.core_classes import BasicModule
from gale.torch_utils import trainable_params
from gale.utils.activs import ACTIVATION_REGISTRY
from gale.utils.shape_spec import ShapeSpec
from gale.utils.structures import IMAGE_CLASSIFIER_HEADS

_logger = logging.getLogger(__name__)

<IPython.core.display.Javascript object>

In [ ]:
# hide
from fastcore.test import *
from gale.utils.logger import setup_logger
from omegaconf import MISSING, DictConfig, OmegaConf

setup_logger()
_logger = logging.getLogger("gale.classification.models.backbones")

<IPython.core.display.Javascript object>

In [ ]:
# export
_all_ = ["IMAGE_CLASSIFIER_HEADS"]

<IPython.core.display.Javascript object>

In [ ]:
# export
class ImageClassificationHead(BasicModule):
    """
    Abstract class for ImageClassification Heads
    """

    _hypers = namedtuple("hypers", field_names=["lr", "wd"])

    @property
    def hypers(self) -> Tuple:
        """
        Returns list of parameters like `lr` and `wd`
        for each param group
        """
        lrs = []
        wds = []

        for p in self.build_param_dicts():
            lrs.append(p["lr"])
            wds.append(p["weight_decay"])
        return self._hypers(lrs, wds)

<IPython.core.display.Javascript object>

In [ ]:
show_doc(ImageClassificationHead.hypers)

<h4 id="ImageClassificationHead.hypers" class="doc_header"><code>ImageClassificationHead.hypers</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Returns list of parameters like `lr` and `wd`
for each param group

<IPython.core.display.Javascript object>

## FullyConnectedHead

In [ ]:
# export
class FullyConnectedHead(ImageClassificationHead):
    """
    Classifier head w/ configurable global pooling and dropout.
    From - https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/layers/classifier.py
    """

    def __init__(
        self,
        input_shape: ShapeSpec,
        num_classes: int,
        pool_type: str = "avg",
        drop_rate: float = 0.0,
        use_conv: bool = False,
        lr: float = 2e-03,
        wd: float = 0,
        filter_wd: bool = False,
    ):
        super(FullyConnectedHead, self).__init__()
        self.drop_rate = drop_rate
        in_planes = input_shape.channels
        # fmt: off
        self.global_pool, num_pooled_features = _create_pool(in_planes, num_classes, pool_type, use_conv=use_conv)
        # fmt: on
        self.fc = _create_fc(num_pooled_features, num_classes, use_conv=use_conv)
        self.flatten_after_fc = use_conv and pool_type

        store_attr("lr, wd, filter_wd")

    def forward(self, x):
        x = self.global_pool(x)
        if self.drop_rate:
            x = F.dropout(x, p=float(self.drop_rate), training=self.training)
        x = self.fc(x)
        return x

    def build_param_dicts(self) -> Any:
        # fmt: off
        if self.filter_wd:
            ps = filter_weight_decay(self, lr=self.lr, weight_decay=self.wd)
        else:
            ps = [{"params": trainable_params(self),"lr": self.lr,"weight_decay": self.wd}]
        return ps

<IPython.core.display.Javascript object>

Arguments to `FullyConnectedHead`:
- `input_shape` (ShapeSpec): input shape
- `num_classes` (int): Number of classes for the head.
- `pool_type` (str): The pooling layer to use. Check [here](https://github.com/rwightman/pytorch-image-models/blob/9a1bd358c7e998799eed88b29842e3c9e5483e34/timm/models/layers/adaptive_avgmax_pool.py#L79).
-  `drop_rate` (float): If >0.0 then applies dropout between the pool_layer and the fc layer.
- `use_conv` (bool): Use a convolutional layer as the final fc layer.
- `lr` (float): Learning rate for the modules.
- `wd` (float): Weight decay for the modules.
- `filter_wd` (bool): Filter out `bias`, `bn` from `weight_decay`.

In [ ]:
input_shape = ShapeSpec(channels=512)
tst = FullyConnectedHead(input_shape, 10)
tst

FullyConnectedHead(
  (global_pool): SelectAdaptivePool2d (pool_type=avg, flatten=True)
  (fc): Linear(in_features=512, out_features=10, bias=True)
)

<IPython.core.display.Javascript object>

In [ ]:
# hide
o = tst(torch.randn(2, 512, 2, 2))
o.shape

torch.Size([2, 10])

<IPython.core.display.Javascript object>

### Dataclass

In [ ]:
# export
@dataclass
class FCHeadDataClass:
    """
    Base config for `FullyConnectedHead`
    """

    num_classes: int = MISSING
    pool_type: str = "avg"
    drop_rate: float = 0.0
    use_conv: bool = False
    lr: float = 1e-03
    wd: float = 0.0
    use_conv: bool = False

<IPython.core.display.Javascript object>

In [ ]:
input_shape = ShapeSpec(channels=512)
c = OmegaConf.structured(FCHeadDataClass(num_classes=10))
tst = FullyConnectedHead.from_config_dict(c, input_shape=input_shape)
tst

FullyConnectedHead(
  (global_pool): SelectAdaptivePool2d (pool_type=avg, flatten=True)
  (fc): Linear(in_features=512, out_features=10, bias=True)
)

<IPython.core.display.Javascript object>

## FastaiHead

In [ ]:
# export
class FastaiHead(ImageClassificationHead):
    """
    Model head that takes `in_planes` features, runs through `lin_ftrs`, and out `num_classes` classes.

    From -
    https://github.com/fastai/fastai/blob/8b1da8765fc07f1232c20fa8dc5e909d2835640c/fastai/vision/learner.py#L76
    """

    def __init__(
        self,
        input_shape: ShapeSpec,
        num_classes: int,
        act: str = "ReLU",
        lin_ftrs: Optional[List] = None,
        ps: Union[List, int] = 0.5,
        concat_pool: bool = True,
        first_bn: bool = True,
        bn_final: bool = False,
        lr: float = 2e-03,
        wd: float = 0,
        filter_wd: bool = False,
    ):
        super(FastaiHead, self).__init__()
        in_planes = input_shape.channels
        pool = "catavgmax" if concat_pool else "avg"
        pool, nf = _create_pool(in_planes, num_classes, pool, use_conv=False)

        # fmt: off
        lin_ftrs = [nf, 512, num_classes] if lin_ftrs is None else [nf] + lin_ftrs + [num_classes]
        # fmt: on

        bns = [first_bn] + [True] * len(lin_ftrs[1:])

        ps = L(ps)

        if len(ps) == 1:
            ps = [ps[0] / 2] * (len(lin_ftrs) - 2) + ps

        act = ifnone(act, "ReLU")
        # fmt: off
        actns = [ACTIVATION_REGISTRY.get(act)(inplace=True)] * (len(lin_ftrs) - 2) + [None]
        if bn_final:
            actns[-1] = ACTIVATION_REGISTRY.get(act)(inplace=True)
        # fmt: on

        self.layers = [pool]

        for ni, no, bn, p, actn in zip(lin_ftrs[:-1], lin_ftrs[1:], bns, ps, actns):
            self.layers += nn.Sequential(
                nn.BatchNorm1d(ni), nn.Dropout(p), nn.Linear(ni, no, bias=not bns), actn
            )

        if bn_final:
            self.layers.append(nn.BatchNorm1d(lin_ftrs[-1], momentum=0.01))
        self.layers = nn.Sequential(*[l for l in self.layers if l is not None])

        store_attr("lr, wd, filter_wd")

    def forward(self, xb: torch.Tensor) -> Any:
        return self.layers(xb)

    def build_param_dicts(self) -> Any:
        if self.filter_wd:
            ps = filter_weight_decay(self.layers, lr=self.lr, weight_decay=self.wd)
        else:
            # fmt: off
            ps = [{"params": trainable_params(self.layers),"lr": self.lr,"weight_decay": self.wd}]
            # fmt: on
        return ps

<IPython.core.display.Javascript object>

The head begins with `AdaptiveConcatPool2d` if `concat_pool=True` otherwise, it uses traditional average pooling. Then it uses a Flatten layer before going on blocks of `BatchNorm`, `Dropout` and `Linear` layers.

Those blocks start at `in_planes`, then every element of `lin_ftrs` (defaults to [512]) and end at `num_classes`. `ps` is a list of probabilities used for the dropouts (if you only pass 1, it will use half the value then that value as many times as necessary).

Arguments to `FastaiHead`:
- `input_shape` (ShapeSpec): input shape
- `num_classes` (int): Number of classes for the head.
- `act` (str): name of the activation function to use. If None uses the default activations else the name must be in ACTIVATION_REGISTRY. Activation layers are used after every block (`BatchNorm`, `Dropout` and `Linear` layers) if it is not the last block.
- `lin_ftrs` (List): Features of the Linear layers. (defaults to [512])
- `ps` (List): list of probabilities used for the dropouts.
- `concat_pool` (bool): Wether to use `AdaptiveConcatPool2d` or `AdaptiveAveragePool2d`.
- `first_bn` (bool): BatchNorm Layer after pool.
- `bn_final` (bool): Final Layer is BatchNorm.
- `lr` (float): Learning rate for the modules.
- `wd` (float): Weight decay for the modules.
- `filter_wd` (bool): Filter out `bias`, `bn` from `weight_decay`.

In [ ]:
input_shape = ShapeSpec(channels=512)
tst = FastaiHead(input_shape=input_shape, num_classes=10)
tst

FastaiHead(
  (layers): Sequential(
    (0): SelectAdaptivePool2d (pool_type=catavgmax, flatten=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.25, inplace=False)
    (3): Linear(in_features=1024, out_features=512, bias=False)
    (4): ReLU(inplace=True)
    (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Dropout(p=0.5, inplace=False)
    (7): Linear(in_features=512, out_features=10, bias=False)
  )
)

<IPython.core.display.Javascript object>

In [ ]:
# hide
o = tst(torch.randn(2, 512, 2, 2))
o.shape

torch.Size([2, 10])

<IPython.core.display.Javascript object>

### Dataclass

In [ ]:
# export
@dataclass
class FastaiHeadDataClass:
    num_classes: int = MISSING
    act: str = "ReLU"
    lin_ftrs: Optional[List] = None
    ps: Any = 0.5
    concat_pool: bool = True
    first_bn: bool = True
    bn_final: bool = False
    lr: float = 0.002
    wd: float = 0
    filter_wd: bool = False

<IPython.core.display.Javascript object>

In [ ]:
conf = OmegaConf.structured(FastaiHeadDataClass(num_classes=10))
tst = FastaiHead.from_config_dict(conf, input_shape=input_shape)
tst

FastaiHead(
  (layers): Sequential(
    (0): SelectAdaptivePool2d (pool_type=catavgmax, flatten=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.25, inplace=False)
    (3): Linear(in_features=1024, out_features=512, bias=False)
    (4): ReLU(inplace=True)
    (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Dropout(p=0.5, inplace=False)
    (7): Linear(in_features=512, out_features=10, bias=False)
  )
)

<IPython.core.display.Javascript object>

## Export-

In [ ]:
# hide
notebook2script("04a_classification.models.heads.ipynb")

Converted 04a_classification.models.heads.ipynb.


<IPython.core.display.Javascript object>